# Check missings:  images and annotations
version: 1

info: 


author: nuno costa

In [68]:
from annotate_v5 import *
import platform 
import numpy as np
import pandas as pd
from IPython.display import Image, display
import copy
import os
from shutil import copyfile
import matplotlib.pyplot as plt
from matplotlib.image import imread
from matplotlib.patches import Rectangle
import random

In [69]:
#Define root dir dependent on OS
rdir_dsets='D:/external_datasets/' #WARNING: DATASETS ROOT is OK?
rdir='D:/external_datasets/MOLA/' 
if str(platform.platform()).find('linux')>-1:
    dirdir_dsets=rdir_dsets.replace('D:/','/mnt/d/')
    rdir=rdir.replace('D:/','/mnt/d/')
print('OS: {}'.format(platform.platform()))
print('root datasets dir: {}'.format(rdir_dsets))
print('root dir: {}'.format(rdir))

OS: Windows-10-10.0.21296-SP0
root datasets dir: D:/external_datasets/
root dir: D:/external_datasets/MOLA/


In [70]:
#jsonfile
injsonfile="splitann_mola_fix_equal_reorder_cleanclass_cleanimg/val" #"split_mola_fix_equal/test"
molajson =  json.load(open(rdir+'annotations/'+injsonfile+'.json'))
for k in molajson:
    print(k, len(molajson[k]))

info 5
licenses 9
categories 373
videos 1488
images 177936
tracks 8132
segment_info 0
annotations 247051
datasets 2


## 1. Import ids
#### #NOTE: work with ids and index so you can use numpy for faster operations

In [71]:
# datasets name and id
dset_l=[]
dset_l_id=[]
try:
    for d in molajson['datasets']:
        dset_l.append(d['name'])
        dset_l_id.append(d['id'])
except: #manually add for example for only COCO
    dset_l=['COCO']
    dset_l_id=[1]
print(dset_l, dset_l_id)

['COCO', 'TAO'] [1, 2]


In [72]:
# categories name and id
cat_l=[]
cat_l_id=[]
cat_l_dset=[]
for c in molajson['categories']:
    cat_l.append(c['name'])
    cat_l_id.append(c['id'])
    try:
        cat_l_dset.append(dset_l[c['dataset']-1]) # dset_l index is same as id-1
    except:
        cat_l_dset.append(dset_l[0])
#print(cat_l_id)

In [73]:
# images filepath and id
img_l=[]
img_l_id=[]
for c in molajson['images']:
    img_l.append(c['file_name'])
    img_l_id.append(c['id'])

In [74]:
# annotations category_id, image_id, bbox, and dataset
ann_catid=[]
ann_imgid=[]
ann_bbox=[]
ann_dset=[]
for an in tqdm(molajson['annotations']):
    ann_catid.append(an['category_id'])
    ann_imgid.append(an['image_id'])
    ann_bbox.append(an['bbox'])
    try:
        ann_dset.append(an['dataset'])
    except:
        ann_dset.append(dset_l_id[0])

100%|████████████████████████████████████████████████████████████████████| 247051/247051 [00:00<00:00, 988260.55it/s]


## 2. Find missings cat_ids
missings example
categories= [{name:cow, id:1, dataset:1},...,{name:cow, id:200, dataset:2},...,{name:cow, id:101, dataset:3}]}

In [75]:
#checks #TODO: SORT alphabetically
checks_l=[]
checks_l_catid=[]
checks_l_catdset=[]
check_method="all_cats"
if check_method=="all_cats": #Do for all category names, even with equal 
    checks_l=cat_l
    checks_l_catid=[[id] for id in cat_l_id]
    checks_l_catdset=[[dset] for dset in cat_l_dset]
    

print(checks_l[0:5])
print(checks_l_catid[0:5])
print(checks_l_catdset[0:5])
print(len(checks_l))
print(len(checks_l_catid))
print(len(checks_l_catdset))

['person', 'bicycle', 'car', 'motorcycle', 'airplane']
[[1], [2], [3], [4], [5]]
[['COCO'], ['COCO'], ['COCO'], ['COCO'], ['COCO']]
373
373
373


In [76]:
# get annotations checks
ann_catid_np=np.array(ann_catid)
ann_imgid_np=np.array(ann_imgid)
ann_bbox_np=np.array(ann_bbox)
ann_dset_np=np.array(ann_dset)
checks_l_imgid=[]
checks_l_bbox=[]
checks_l_dset=[]
for catids in tqdm(checks_l_catid):
    l_imgid=[]
    l_bbox=[]
    l_dset=[]
    for catid in catids:
        ann_idx = np.where(ann_catid_np==catid)[0].tolist() #annotation index of ids
        l_imgid.append(ann_imgid_np[ann_idx].tolist())
        l_bbox.append(ann_bbox_np[ann_idx].tolist())
        l_dset.append(ann_dset_np[ann_idx].tolist())
    checks_l_imgid.append(l_imgid)
    checks_l_bbox.append(l_bbox)
    checks_l_dset.append(l_dset)

100%|█████████████████████████████████████████████████████████████████████████████| 373/373 [00:01<00:00, 218.00it/s]


## 3. Classes|categories to check w/ EXCEL report

In [77]:
#INIT VARS
method="save_images" #"save_images": to save new images and excel for manual inspection; "": already saved don't need to repeat and the excel are
datadir="checks/"+injsonfile+"/" #root folder to save check method . #WARNING checks/original json that was used to save images and excel
folder=check_method+'/' #folder to save images and exel 
showimage=False #show images
startidx=0 # start index of image to save from each dataset
imgnr=3 # total number of images to save from each dataset
imgstep='random' # step between images: int | 'random' - int steps between images; 'rand' gets random list
#paths
path=os.path.join(rdir,datadir,folder) #path to folder
assure_path_exists(path)
excelpath=path+check_method+"_report_v1.xlsx"#path+check_method+"_report.xlsx"#path to excel

In [78]:
#METHODS
if method=="save_images": # save images and excel to folder for manual edit
    df=pd.DataFrame({'checks_l': checks_l,'checks_l_catid': checks_l_catid, 'checks_l_catdset': checks_l_catdset })
    df['annotations_missing'] = np.empty((len(df), 0)).tolist()
    df['images_missing'] = np.empty((len(df), 0)).tolist()
    #save image for each check
    for i, check in enumerate(tqdm(checks_l)): #run for each check category
        firstclass=check
        if isinstance(firstclass, list): firstclass=firstclass[0] #first class
        print('\n>> '+firstclass+'...') #class
        classpath=os.path.join(path, firstclass) # path to folder for images of  firstclass
        classpath=parse_path(classpath)+'/' #make it a folder
        assure_path_exists(classpath)
        df=save_imgs(df, rdir_dsets, classpath, i, dset_l, checks_l, checks_l_catid, checks_l_bbox, checks_l_dset,
              checks_l_imgid, img_l, img_l_id, startidx=startidx, imgnr=imgnr, imgstep=imgstep, showimage=showimage)    
    df.to_excel(excelpath, index=False)   

In [79]:
#IMPORT EXCEL MANUAL EDIT #WARNING: CHECK EXCEL FIRST (#NOTE: donte use classes with missing annotations and images)
df=pd.read_excel(excelpath)
display(df)        

,mixers_l,mixers_l_catid,mixers_l_catdset,classtomix_l,classtomix_l_catid,rules,annotations_missing,images_missing,Unnamed: 8,aggressive_keywords,class,notes
0,person,[1],['COCO'],aggressive,"[35, 43, 44, 45, 77, 105, 113, 131, 165, 190, ...",To fix classes: 1) You need to fill the column...,[0],[0],NaN,knife,"knife(44), butcher_knife(113)",Só adicionei ids com annotations
1,bicycle,[2],['COCO'],person,[1],NaN,[0],[0],NaN,weapon,bow_(weapon)(105),NaN
2,car,[3],['COCO'],bicycle,[2],NaN,[0],[0],NaN,gun,gun(190),NaN
3,motorcycle,[4],['COCO'],car,[3],NaN,[0],[0],NaN,bat,baseball bat(35),NaN
4,airplane,[5],['COCO'],motorcycle,[4],NaN,[0],[0],NaN,fork,fork(43),NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
368,water_ski,[369],['TAO'],NaN,NaN,NaN,[0],[0],NaN,NaN,NaN,NaN
369,wig,[370],['TAO'],NaN,NaN,NaN,[0],[0],NaN,NaN,NaN,NaN
370,windshield_wiper,[371],['TAO'],NaN,NaN,NaN,[0],[0],NaN,NaN,NaN,NaN
371,wineglass,[372],['TAO'],NaN,NaN,NaN,[0],[0],NaN,NaN,NaN,NaN
